In [ ]:
import pandas as pd
import datetime
import requests
import sys


errored = []
op_contract_add = '0x4200000000000000000000000000000000000042'
api_key = "optimistic.etherscan-api"
url = "https://api-optimistic.etherscan.io/api"
random_state = 27

In [ ]:
data = pd.read_csv('data/optimism_airdrop.csv')

sample = data.sample(n=7729, random_state=random_state)
#sample.reset_index(drop=True,inplace=True)

not_in_sample = ~data.isin(sample).all(1)
new_data = data[not_in_sample]
more_data = new_data[:13000]

In [ ]:
def wei_to_op(wei):
    return int(wei) / 10**18

In [ ]:
def get_balance(address):
    """
    Funtion return the Eth balance of an a given address

    >>> get_balance('0x0000000000000000000000000000000000000000')
    0.00897765567

    """
    params = {
        "module": "account",
        "action": "balance",
        "address": address,
        "tag": "latest",
        "apikey": api_key
    }

    # Make a GET request with the URL and query parameters
    response = requests.get(url, params=params)

    # Print the response content
    return round(wei_to_op(response.json()['result']),5)

In [ ]:
def get_Erc20_transaction_history(address,opt=True):
    if opt == True:
        params = {
            "module": "account",
            "action": "tokentx",
            "contractaddress" : op_contract_add,
            "address": address,
            "offset": 100,
            "startblock": 0,
            "endblock": 99999999,
            "sort": "asc",
            "apikey": api_key
        }

    else:
        params = {
            "module": "account",
            "action": "tokentx",
            "address": address,
            "page": 1,
            "offset": 100,
            "startblock": 0,
            "endblock": 99999999,
            "sort": "asc",
            "apikey": api_key
        }

    
    response = requests.get(url, params=params).json()
    return response['result']


In [ ]:
def locate_airdrop_amount(data,address):
    amount = data['total_op'].loc[data['address']==address].item()
    return amount

In [ ]:
def get_sell_off_date(history:list[dict], op_amount:int, address:str):
    today = datetime.date.today()
    for transaction in history:
        value = wei_to_op(transaction['value'])
        if round(value ,8) == round(op_amount,8):
            time = int(transaction['timeStamp'])
            initial_date = datetime.datetime.fromtimestamp(time).strftime("%Y-%m-%d %H:%M:%S")
            break
    else:
        print(f"No transaction found with value {op_amount} OP for address {address}")
        errored.append(address)
        return [address,'error','error','error']
    
    expended_op = 0
    for transaction in history:
        timestamp = int(transaction['timeStamp'])
        token_symbol = transaction['tokenSymbol']
        value = int(transaction['value'])
        
        if timestamp >= time and token_symbol == "OP" and transaction['from'] == address:
            expended_op += value
            percentage_spent = round(wei_to_op(expended_op) / op_amount * 100, 2)
            
            if percentage_spent >= 100:
                percentage_spent = 100
                
                out_date = datetime.datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')
                delta = datetime.datetime.strptime(out_date, '%Y-%m-%d %H:%M:%S') - datetime.datetime.strptime(initial_date, '%Y-%m-%d %H:%M:%S')
                days = delta.days
                row = [address, days, wei_to_op(expended_op), percentage_spent]
                return row
            
    percentage_spent = round(wei_to_op(expended_op) / op_amount * 100, 2)
    
    if percentage_spent >= 100:
        percentage_spent = 100
    
    initial_date = datetime.datetime.fromtimestamp(time)
    u = datetime.date(2023, 4, 18)
    delta = u - initial_date.date()
    days = delta.days

    row = [address, days, wei_to_op(expended_op), percentage_spent]
    
    return row


In [ ]:
def calculate_txn_day_ratio(txn_history):
    # Convert timestamps to datetime objects and sort transactions by timestamp
    transactions = sorted(
        [datetime.datetime.fromtimestamp(int(txn['timeStamp'])) for txn in txn_history]
    )
    
    # Calculate the number of days between the earliest and latest transactions
    delta_days = (transactions[-1] - transactions[0]).days
    
    # Calculate the transaction-day ratio
    if delta_days == 0:
        txn_day_ratio = len(transactions)
    else:
        txn_day_ratio = len(transactions) / delta_days
        
    return round(txn_day_ratio,3)

In [ ]:
def get_wallet_age(history: list[dict]):
    creation_time = int(history[0]['timeStamp'])
    creation_date = datetime.datetime.fromtimestamp(creation_time).date()
    current_date = datetime.date(2023, 4, 18)  # Set current date to 18th of April 2023
    wallet_age = (current_date - creation_date).days
    return wallet_age

In [ ]:
def to_and_from(history:list[dict],address):
    from_count = 0
    to_count = 0
    for transactions in history:
        if transactions['from'] == address:
            from_count +=1
        else:
            to_count += 1
    return from_count, to_count

In [ ]:
def Previous_drop_participant(history:list[dict]):
    Participant = False
    for transaction in history:
        if transaction['from'] == '0xfedfaf1a10335448b7fa0268f56d2b44dbd357de':
            Participant = True

    return Participant

In [ ]:
info = []
import time
def main(data):
    for i, address in enumerate(data['address']):
        Optimism_transaction_dict = get_Erc20_transaction_history(address)

        Erc_20_history = get_Erc20_transaction_history(address,False)
        
        op = locate_airdrop_amount(data,address)
        
        column = get_sell_off_date(Optimism_transaction_dict,op,address)
        
        tdr  = calculate_txn_day_ratio(Erc_20_history)
        
        eth_balance = get_balance(address)

        wallet_age = get_wallet_age(Erc_20_history)

        from_count, to_count = to_and_from(Erc_20_history,address)

        Past_participant = Previous_drop_participant(Optimism_transaction_dict)
        
        column.insert(1,eth_balance)
        
        column.insert(-2,op)
        
        column.append(tdr)

        column.append(wallet_age)

        column.append(from_count)

        column.append(to_count)

        column.append(Past_participant)
        
        info.append(column)
        
        # Print iteration count every 100 iterations
        if i % 5 == 0:
            print(f"Iteration: {i}")
            time.sleep(1)


In [ ]:
main(new_data[6817:])

In [ ]:
columns = ['address','balance','Days_held','Amount_Received(OP)','Amount_Sold(OP)','percentage_spent','TDR','Wallet_Age(Days)','From_Count','To_Count','Past_participant']
query = pd.DataFrame(info,columns=columns)

query.to_csv('queried_data/more_data1.csv',index=False)